In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

In [2]:
# sportify specific
from data.metrics_track import r_precision, cosine_sim_closest, cosine_sim_top, NDCG
from visualization.plot_utils import write_latex_table
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.audio_features import AudioFeatures
from features.tracks_info import TrackInfo
from collections import OrderedDict

# Load track data

In [3]:
tracks = TrackInfo(subset='5k-')


NameError: name 'get_playlist' is not defined

In [11]:
pid=194
turi_seed,auri_seed, turi_true,auri_true, turi_pool,auri_pool = tracks.get_playlist_pooltest(pid=pid,k=25)



In [6]:
k_range = [1, 5, 10, 25, 100]
max_picks = 500
# load playlist
pid = 194
pl_turi,pl_auri = tracks.get_playlist(pid)
n_songs = len(pl_turi)

In [ ]:
X_t = af.transform()

In [4]:
# audio features
af = AudioFeatures(subset='5k-')
X = af.transform()
n_cols = X.shape[1]

In [7]:
xp = af.transform(pl_turi)
xp.shape

(130, 9)

In [17]:
suggest_df = pd.DataFrame()
n_top = 10
x_pred = OrderedDict()
stats = []
print('Test on playlist {:d}'.format(pid))
for k in k_range:
    x_pred[k] = []
    k_picks = int(max_picks / k)
    for i in range(k):
        x_query = xp[i - 1, :]
        ind, sim = cosine_sim_closest(X, x_query, k_picks)
        x_pred[k] = x_pred[k] + list(ind)
    subset_ids = pl_ints[k:]
    result = OrderedDict()
    result['k'] = k
    result['r precision'] = r_precision(subset_ids, x_pred[k])
    result['NDGC'] = NDCG(subset_ids, x_pred[k])
    stats.append(result)

sdf = pd.DataFrame(stats)
print(sdf)

write_latex_table(sdf, 'metrics', adir='.', render=True)

Test on playlist 194
     k  r precision      NDGC
0    1     0.023256  0.463649
1    5     0.024000  0.269681
2   10     0.033333  0.228421
3   25     0.009524  0.000000
4  100     0.000000  0.000000
